# Les fondammentaux de Spark

## Spark, quesaco?

Apache Spark est un moteur unifié conçu pour le traitement de données distribué à grande échelle, dans des locaux de centres de données ou dans le nuage.
Spark fournit un stockage en mémoire pour les calculs intermédiaires, ce qui le rend beaucoup plus rapide que Hadoop MapReduce. Il intègre des bibliothèques avec des API composables pour l'apprentissage machine (MLlib), du SQL pour les requêtes interactives (Spark SQL), du streaming (Structured Streaming) pour l'interaction avec les données en temps réel et du traitement de graphe (GraphX).
La philosophie de conception de Spark s'articule autour de quatre caractéristiques clés :
- Vitesse
- Facilité d'utilisation
- Modularité 
- Extensibilité

### Vitesse

Spark a poursuivi l'objectif de la vitesse de plusieurs manières. Tout d'abord, sa mise en œuvre interne bénéficie énormément des énormes progrès réalisés récemment par l'industrie du matériel informatique pour améliorer le prix et les performances des unités centrales et de la mémoire. Les serveurs actuels sont bon marché, avec des centaines de gigaoctets de mémoire, des cœurs multiples et un système d'exploitation Unix sous-jacent qui tire parti du multithreading et du traitement parallèle efficaces. Le cadre est optimisé pour tirer profit de tous ces facteurs.

Ensuite, Spark construit ses calculs de requêtes sous forme de graphe acyclique dirigé (DAG) ; son programmateur DAG et son optimiseur de requêtes construisent un graphe de calcul efficace qui peut généralement être décomposé en tâches qui sont exécutées en parallèle par les travailleurs de la grappe. Et troisièmement, son moteur d'exécution physique, Tungsten, utilise la génération de code en phase complète pour générer un code compact pour l'exécution.
Avec tous les résultats intermédiaires conservés en mémoire et ses entrées/sorties limitées sur disque, cela lui donne un énorme avantage en termes de performances.

### Facilite d'utilisation

Spark atteint la simplicité en fournissant une abstraction fondamentale d'une structure de données logique simple appelée "Resilient Distributed Dataset" (RDD) sur laquelle toutes les autres abstractions de données structurées de niveau supérieur, telles que les DataFrames et les Datasets, sont construites. En fournissant un ensemble de transformations et d'actions sous forme d'opérations, Spark propose un modèle de programmation simple que vous pouvez utiliser pour construire de grandes applications de données dans des langages familiers.

### Modularite

Les opérations de Spark peuvent être appliquées à de nombreux types de charges de travail et être exprimées dans n'importe lequel des langages de programmation pris en charge : Scala, Java, Python, SQL et R. Spark offre des bibliothèques unifiées avec des APIs bien documentées qui comprennent les modules suivants comme composants de base : Spark SQL, Spark Structured Streaming, Spark MLlib, et GraphX, combinant toutes les charges de travail fonctionnant sous un seul moteur. Nous examinerons tous ces éléments de plus près dans la prochaine section.
Vous pouvez écrire une seule application Spark qui peut tout faire - pas besoin de moteurs distincts pour des charges de travail disparates, pas besoin d'apprendre des APIs séparées. Avec Spark, vous disposez d'un moteur de traitement unifié pour vos charges de travail.

### Extensibilite

Spark se concentre sur son moteur de calcul rapide et parallèle plutôt que sur le stockage. Contrairement à Apache Hadoop, qui comprenait à la fois le stockage et le calcul, Spark découple les deux. Cela signifie que vous pouvez utiliser Spark pour lire des données stockées dans une myriade de sources - Apache Hadoop, Apache Cassandra, Apache HBase, MongoDB, Apache Hive, SGBDR, et plus - et les traiter en mémoire. Les DataFrameReaders et DataFrame Writers de Spark peuvent également être étendus pour lire des données provenant d'autres sources, telles que Apache Kafka, Google Cloud Storage, Azure Storage, et Amazon S3, dans son abstraction logique de données, sur laquelle elle peut fonctionner.

La communauté des développeurs Spark maintient une liste de paquets Spark tiers dans le cadre de l'écosystème en pleine croissance (voir figure). Ce riche écosystème de paquets comprend des connecteurs Spark pour une variété de sources de données externes, des moniteurs de performance, et plus encore.

![spark_connector.png](images/spark_connector.png)

## Les composants de Spark

Comme le montre la figure suivante, Spark propose quatre composantes distinctes en tant que bibliothèques pour des charges de travail diverses : Spark SQL, Spark MLlib, Spark Structured Streaming et GraphX. Chacun de ces composants est séparé du moteur central de Spark, tolérant aux pannes, en ce sens que vous utilisez des API pour écrire votre application Spark et que Spark la convertit en un DAG qui est exécuté par le moteur central. Ainsi, que vous écriviez votre code Spark en utilisant les API structurées en Java, R, Scala, SQL ou Python, le code sous-jacent est décomposé en un bytecode très compact qui est ex-coupé dans les JVM des travailleurs de l'ensemble du cluster.

![spark_components.png](images/spark_components.png)

Nous examinerons chacun de ces modules dans les chapitres a venir.

## L'exécution distribuée d'Apache Spark

À un niveau élevé de l'architecture, une application Spark consiste en un programme pilote (driver) qui est responsable de l'orchestration des opérations parallèles sur le cluster Spark. Le driver accède aux composants distribués dans le cluster - les exécuteurs Spark et le gestionnaire du cluster - par le biais d'une session Spark.

![spark_orchestration.png](images/spark_orchestration.png)

### Le Spark driver

En tant que partie de l'application Spark responsable de l'instanciation d'une session Spark, le drive a plusieurs rôles : 
* il communique avec le gestionnaire du cluster ;
* il demande des ressources (CPU, mémoire, etc.) au gestionnaire du cluster pour les exécuteurs Spark (JVM) ; 
* et il transforme toutes les opérations Spark en calculs DAG, les programme et distribue leur exécution sous forme de tâches entre les exécuteurs Spark. 

Une fois les ressources allouées, il communique directement avec les exécuteurs.

### La SparkSession

Dans Spark 2.0, la SparkSession est devenue un conduit unifié pour toutes les opérations et les données Spark. Non seulement elle a intégré les points d'entrée précédents de Spark comme SparkContext, SQLContext, HiveContext, SparkConf et StreamingContext, mais elle a également rendu le travail avec Spark plus simple et plus facile.

Grâce à ce conduit unique, vous pouvez créer les paramètres d'exécution de la JVM, définir des cadres et des ensembles de données, lire à partir de sources de données, accéder aux métadonnées du catalogue et émettre des requêtes Spark SQL. SparkSession fournit un point d'entrée unique et unifié à toutes les fonctionnalités de Spark.

Dans une application Spark autonome (ou standalone), vous pouvez créer une session SparkSession en utilisant l'une des API de haut niveau dans le langage de programmation de votre choix. Dans le shell Spark, la session Spark est créée pour vous, et vous pouvez y accéder via une variable globale appelée spark ou sc.

Examinons un exemple avec Scala : 

In [0]:
import org.apache.spark.sql.SparkSession

Build SparkSession
val spark = SparkSession
    .builder
    .config("spark.sql.shuffle.partitions", 6)
    .getOrCreate()
// Use the session to read JSON
// val people = spark.read.json("...")
// Use the session to issue a SQL query
// val resultsDF = spark.sql("SELECT city, pop, state, zip FROM table_name")

cmd0.sc:3: not found: value Build
val res0_1 = Build SparkSession
             ^cmd0.sc:1: object apache is not a member of package org
import org.apache.spark.sql.SparkSession
           ^cmd0.sc:4: not found: value SparkSession
val spark = SparkSession
            ^Compilation Failed

: 

### Le cluster manager

Le gestionnaire du cluster est responsable de la gestion et de l'allocation des ressources pour le cluster de nœuds sur lequel tourne votre application Spark. Actuellement, Spark prend en charge quatre gestionnaires de clusters : le gestionnaire de cluster autonome intégré, Apache Hadoop YARN, Apache Mesos et Kubernetes.

### Les modes de deploiement

Une caractéristique intéressante de Spark est sa prise en charge d'une myriade de modes de déploiement, ce qui permet à Spark de fonctionner dans différentes configurations et différents environnements. Comme l'homme du cluster est indifférent à l'endroit où il fonctionne (tant qu'il peut gérer les exécuteurs de Spark et répondre aux demandes de ressources), Spark peut être déployé dans certains des environnements les plus populaires, tels que Apache Hadoop YARN et Kubernetes, et peut fonctionner en différents modes. Le tableau suivant résume les modes de déploiement disponibles.

![spark_cluster_managers.png](images/spark_cluster_managers.png)

### Les Données distribuées et partitions

Les données physiques réelles sont réparties sur le stockage sous forme de partitions résidant soit dans le HDFS, soit dans le stockage cloud. Alors que les données sont distribuées sous forme de partitions dans le cluster physique, Spark traite chaque partition comme une abstraction logique de données de haut niveau - comme un DataFrame en mémoire. Bien que cela ne soit pas toujours possible, chaque exécuteur Spark se voit de préférence attribuer une tâche qui lui demande de lire la partition la plus proche de lui dans le réseau, en observant la localisation des données.

![spark_partitions.png](images/spark_partitions.png)

Le partitionnement permet un parallélisme efficace. Un schéma distribué de découpage des données en morceaux ou en partitions permet aux exécuteurs de Spark de ne traiter que les données qui leur sont proches, ce qui minimise la bande passante du réseau. En d'autres termes, le cœur de chaque exécuteur se voit attribuer sa propre partition de données sur laquelle il doit travailler (voir figure suivante).

![spark_executor_partitions.png](images/spark_executor_partitions.png)

Par exemple, le extrait de code suivant divisera les données physiques stockées dans les grappes en huit partitions, et chaque exécuteur obtiendra une ou plusieurs partitions à lire dans sa mémoire :

Et ce code créera une DataFrame de 10000 entiers répartis sur huit parties en mémoire :

## Spark sur Jupyter Notebook

### Ammonite

Ammonite vous permet d'utiliser le langage Scala à des fins de scripting : dans le REPL, comme scripts, comme bibliothèque à utiliser dans des projets existants, ou comme shell de systèmes autonomes. Il est compose par les modules : `Ammonite-REPL`, `Ammonite-Ops`, `Scala Scripts` et `Ammonite-Shell`: 

(Cf. live demo pour leur utilisation)

#### Ammonite-REPL

C'est un REPL Scala modernisé. Avec la coloration syntaxique, l'édition multi-lignes, la possibilité de charger des artefacts maven directement dans le REPL, et de nombreuses autres améliorations de la qualité de vie qui manquaient dans le REPL Scala par défaut.

La figure suivante montre a quoi cela ressemble:
![ammonite_repl.png](images/ammonite_repl.png)

#### Scala Scripts

Il permet une programmation légère en Scala. Il vous permet de créer des scripts que vous pouvez exécuter facilement en ligne de commande, sans avoir à mettre en place un "projet" ou à attendre les lents délais de démarrage de SBT.

![ammonite_scripts](images/ammonite_scripts.gif)

#### Ammonite-Ops

C'est une bibliothèque de système de fichiers solide pour Scala. Il vous permet de gérer facilement le système de fichiers à partir de vos projets ou applications Scala existants, aussi facilement que vous le feriez à partir d'un script Bash ou Python.

#### Ammonite-Shell

C'est un remplacement moderne du système Bash. Il fournit un shell système dans le langage Scala de haut niveau, vous permettant de mélanger de façon transparente les opérations système avec du vrai code sans les tracas ou la frustration d'essayer d'écrire du code complexe en Bash.

![ammonite_shell.png](images/ammonite_shell.png)

### Almond

Almond enveloppe Ammonite dans un noyau de Jupyter, vous donnant toutes ses caractéristiques et subtilités, y compris une jolie impression personnalisable, des importations magiques, une gestion avancée des dépendances, son API, directement de Jupyter.

Cela permet également de copier facilement certains codes des ordinateurs portables vers les scripts Ammonite, et vice versa.

In [1]:
// Expressions are computable statements.
1 + 1

res0: Int = 2

In [2]:
// You can output results of expressions using println.
println(1) // 1
println(1 + 1) // 2
println("Hello!") // Hello!
println("Hello," + " world!") // Hello, world!

1
2
Hello!
Hello, world!


### Almond-spark

Almond est livré avec un module d'intégration Spark appelé almond-spark, qui vous permet de vous connecter à un cluster Spark et d'effectuer des calculs Spark de manière interactive à partir de Jupyter. 

Il est basé sur ammonite-spark, ajoutant des caractéristiques spécifiques à Jupyter telles que les barres de progression et l'annulation pour exécuter les calculs Spark.

ammonite-spark gère le chargement de Spark de manière intelligente, et ne dépend pas d'une distribution spécifique de Spark. De ce fait, vous pouvez l'utiliser avec n'importe quelle version de Spark 2.x. La seule limitation est que la version Scala de Spark et le noyau Almond en cours d'exécution doivent correspondre, donc assurez-vous que votre noyau utilise la même version Scala que votre cluster Spark. Spark 2.0.x - 2.3.x nécessite Scala 2.11. Spark 2.4.x supporte à la fois Scala 2.11 et 2.12.

Pour l'utiliser, il suffit d'importer Spark 2.x, la dépendance almond-spark sera ajoutée automatiquement.

In [3]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark

import org.apache.spark.sql._

import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark


import org.apache.spark.sql._

En général, vous souhaitez désactiver le logging afin d'éviter de polluer les sorties de vos cellules.

In [4]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


Créez ensuite une session `SparkSession` à l'aide du `NotebookSparkSessionBuilder` fourni par *almond-spark*.

#### Fonctionnement en mode local

Cela fera fonctionner Spark dans la même JVM que votre noyau.

In [5]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[4]")
    .getOrCreate()
}


Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@7fb86e7f

Lorsque vous exécutez cette fonction, vous devez voir que la sortie de la cellule contient un lien vers l'interface utilisateur de Spark.

Le builder renvoyé par `NotebookSparkSession.builder()` étend celui de `SparkSession.builder()`, de sorte que vous pouvez y appeler `.appName("foo")`, `.config("key", "value")`, etc.

#### Connection à un véritable cluster

Bien sûr, vous pouvez également vous connecter à un véritable cluster Spark. ammonite-spark supporte actuellement les clusters autonomes et les clusters de fils. Les Mesos et Kubernetes ne sont pas encore pris en charge.

#### Utilisation avec un cluster standalone

Il suffit de régler le maître sur spark://... lors de la construction de la session, par exemple.

In [ ]:
// val spark = {
//   NotebookSparkSession.builder()
//     .master("spark://localhost:7077")
//     .config("spark.executor.instances", "4")
//     .config("spark.executor.memory", "2g")
//     .getOrCreate()
// }

Assurez-vous que la version de Spark utilisée pour démarrer le maître et les exécuteurs correspond à celle chargée dans la session du portable (via par exemple import $ivy.`org.apache.spark::spark-sql:X.Y.Z`), et que la machine exécutant le noyau peut accéder / est accessible depuis tous les nœuds du cluster autonome.

#### Utilisation avec un cluster YARN

Réglez le master sur "yarn" lors de la construction de la session, par exemple:

In [ ]:
// val spark = {
//   NotebookSparkSession.builder()
//     .master("yarn")
//     .config("spark.executor.instances", "4")
//     .config("spark.executor.memory", "2g")
//     .getOrCreate()
// }

Maintenant que nous avons une session Spark, nous pouvons obtenir un contexte Spark en effectuant des calculs Spark.

In [4]:
def sc = spark.sparkContext

defined function sc

Ensuite, créez un RDD et effectuez quelques calculs.

In [5]:
val rdd = sc.parallelize(1 to 100000000, 100)

rdd: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at cmd4.sc:1

In [6]:
val n = rdd.map(_ + 1).sum()

sum at cmd5.sc:1

100 / 100

n: Double = 5.00000015E15

#### Synchronisation des dépendances

In [7]:
import $ivy.`org.typelevel::cats-core:1.6.0`

NotebookSparkSession.sync() // cats should be available on workers

import $ivy.$                               


res6_1: SparkSession = org.apache.spark.sql.SparkSession@3ff39280

https://almond.sh/